# Mancala AI Project
### Submitted By: Ethan Meli

The below cells show the implementation of Mancala from HW 7, as well as a demo in which the User is able to play against a random player:

In [1]:
import random
random.seed(109)

In [14]:
class Mancala:
    def __init__(self, pits_per_player=6, stones_per_pit = 4):
        """
        The constructor for the Mancala class defines several instance variables:

        pits_per_player: This variable stores the number of pits each player has.
        stones_per_pit: It represents the number of stones each pit contains at the start of any game.
        board: This data structure is responsible for managing the Mancala board.
        current_player: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        moves: This is a list used to store the moves made by each player. It's structured in the format (current_player, chosen_pit).
        p1_pits_index: A list containing two elements representing the start and end indices of player 1's pits in the board data structure.
        p2_pits_index: Similar to p1_pits_index, it contains the start and end indices for player 2's pits on the board.
        p1_mancala_index and p2_mancala_index: These variables hold the indices of the Mancala pits on the board for players 1 and 2, respectively.
        """
        self.pits_per_player = pits_per_player
        self.board = [stones_per_pit] * ((pits_per_player+1) * 2)  # Initialize each pit with stones_per_pit number of stones 
        self.players = 2
        self.current_player = 1
        self.moves = []
        self.p1_pits_index = [0, self.pits_per_player-1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player+1, len(self.board)-1-1]
        self.p2_mancala_index = len(self.board)-1
        
        # Zeroing the Mancala for both players
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0

    def display_board(self):
        """
        Displays the board in a user-friendly format
        """
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            else:    
                print('{} -> | {} | {} | <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            
        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.current_player == 1 else 'P2'
        print('Turn: ' + turn)
        
    def valid_move(self, pit):
        """
        Function to check if the pit chosen by the current_player is a valid move.
        """
        if self.current_player == 1:
            pit_index = pit - 1
            if self.p1_pits_index[0] <= pit_index <= self.p1_pits_index[1] and self.board[pit_index] > 0:
                return True
        else:
            pit_index = self.p2_pits_index[0] + (pit - 1)
            if self.p2_pits_index[0] <= pit_index <= self.p2_pits_index[1] and self.board[pit_index] > 0:
                return True
        return False
        
    def random_move_generator(self):
        """
        Function to generate random valid moves with non-empty pits for the random player
        """     
        if self.current_player == 1:
            available_pits = [i + 1 for i in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1) if self.board[i] > 0]
        else:
            available_pits = [j - self.p2_pits_index[0] + 1 for j in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1) if self.board[j] > 0]

        return random.choice(available_pits) if available_pits else None
    
    def play(self, pit):
        """
        This function simulates a single move made by a specific player using their selected pit. It primarily performs three tasks:
        1. It checks if the chosen pit is a valid move for the current player. If not, it prints "INVALID MOVE" and takes no action.
        2. It verifies if the game board has already reached a winning state. If so, it prints "GAME OVER" and takes no further action.
        3. After passing the above two checks, it proceeds to distribute the stones according to the specified Mancala rules.

        Finally, the function then switches the current player, allowing the other player to take their turn.
        """
        if not self.valid_move(pit):
            # print("INVALID MOVE")
            if self.current_player == 1:
                self.current_player = 2
            else:
                self.current_player = 1
            return self.board
        
        if self.current_player == 2:
            pit_index = self.p2_pits_index[0] + (pit - 1)
        else:
            pit_index = pit - 1
    
        stones = self.board[pit_index]
        self.board[pit_index] = 0
        position = pit_index
        
        while stones > 0:
            position = (position + 1) % len(self.board)
            self.board[position] += 1
            stones -= 1
        
        if self.current_player == 1 and position in range(*self.p1_pits_index) and self.board[position] == 1:
            opposite_pit = self.p2_pits_index[1] - (position - self.p1_pits_index[0])
            self.board[self.p1_mancala_index] += self.board[opposite_pit] + 1
            self.board[position] = 0
            self.board[opposite_pit] = 0
        elif self.current_player == 2 and position in range(*self.p2_pits_index) and self.board[position] == 1:
            opposite_pit = self.p1_pits_index[1] - (position - self.p2_pits_index[0])
            self.board[self.p2_mancala_index] += self.board[opposite_pit] + 1
            self.board[position] = 0
            self.board[opposite_pit] = 0
        
        self.moves.append((self.current_player, pit))
        
        if self.current_player == 1:
            self.current_player = 2
        else:
            self.current_player = 1
        
        """
        if self.winning_eval():
            print("GAME OVER")
        """
    
        return self.board
    
    def winning_eval(self):
        """
        Function to verify if the game board has reached the winning state.
        Hint: If either of the players' pits are all empty, then it is considered a winning state.
        """
        p1_empty = all(stone == 0 for stone in self.board[self.p1_pits_index[0]:self.p1_pits_index[1]+1])
        p2_empty = all(stone == 0 for stone in self.board[self.p2_pits_index[0]:self.p2_pits_index[1]+1])

        if p1_empty or p2_empty:
            self.board[self.p1_mancala_index] += sum(self.board[self.p1_pits_index[0]:self.p1_pits_index[1]+1])
            self.board[self.p2_mancala_index] += sum(self.board[self.p2_pits_index[0]:self.p2_pits_index[1]+1])

            for i in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1):
                self.board[i] = 0
            for i in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1):
                self.board[i] = 0
            return True
        """
            if self.board[self.p1_mancala_index] > self.board[self.p2_mancala_index]:
                print("Player 1 wins!")
            elif self.board[self.p1_mancala_index] < self.board[self.p2_mancala_index]:
                print("Player 2 wins!")
            else:
                print("It's a tie!")
        """
        return False

### Below is the Demo Game

In [6]:
game_demo = Mancala()
game_demo.display_board()

for i in range(5):
    player_move = int(input("Choose your move (1-6): "))
    game_demo.play(player_move)
    game_demo.display_board()
    print(game_demo.board)

    # Random player move
    if not game_demo.winning_eval():
        random_move = game_demo.random_move_generator()
        print(f"Random Player chose pit: {random_move}")
        game_demo.play(random_move)
        game_demo.display_board()

# Printing the list of moves
print("\nList of moves played:")
for move in game_demo.moves:
    player, pit = move
    print(f"Player {player} selected pit {pit}")

P1               P2
     ____0____     
1 -> | 4 | 4 | <- 6
2 -> | 4 | 4 | <- 5
3 -> | 4 | 4 | <- 4
4 -> | 4 | 4 | <- 3
5 -> | 4 | 4 | <- 2
6 -> |_4_|_4_| <- 1
         0         
Turn: P1


Choose your move (1-6):  2


P1               P2
     ____0____     
1 -> | 4 | 4 | <- 6
2 -> | 0 | 4 | <- 5
3 -> | 5 | 4 | <- 4
4 -> | 5 | 4 | <- 3
5 -> | 5 | 4 | <- 2
6 -> |_5_|_4_| <- 1
         0         
Turn: P2
[4, 0, 5, 5, 5, 5, 0, 4, 4, 4, 4, 4, 4, 0]
Random Player chose pit: 3
P1               P2
     ____1____     
1 -> | 4 | 5 | <- 6
2 -> | 0 | 5 | <- 5
3 -> | 5 | 5 | <- 4
4 -> | 5 | 0 | <- 3
5 -> | 5 | 4 | <- 2
6 -> |_5_|_4_| <- 1
         0         
Turn: P1


Choose your move (1-6):  3


P1               P2
     ____1____     
1 -> | 4 | 5 | <- 6
2 -> | 0 | 5 | <- 5
3 -> | 0 | 5 | <- 4
4 -> | 6 | 0 | <- 3
5 -> | 6 | 4 | <- 2
6 -> |_6_|_5_| <- 1
         1         
Turn: P2
[4, 0, 0, 6, 6, 6, 1, 5, 4, 0, 5, 5, 5, 1]
Random Player chose pit: 2
P1               P2
     ____1____     
1 -> | 4 | 6 | <- 6
2 -> | 0 | 6 | <- 5
3 -> | 0 | 6 | <- 4
4 -> | 6 | 1 | <- 3
5 -> | 6 | 0 | <- 2
6 -> |_6_|_5_| <- 1
         1         
Turn: P1


...

P1               P2
     ____1____     
1 -> | 0 | 6 | <- 6
2 -> | 1 | 6 | <- 5
3 -> | 1 | 6 | <- 4
4 -> | 7 | 1 | <- 3
5 -> | 7 | 0 | <- 2
6 -> |_6_|_5_| <- 1
         1         
Turn: P2
[0, 1, 1, 7, 7, 6, 1, 5, 0, 1, 6, 6, 6, 1]
Random Player chose pit: 5
P1               P2
     ____2____     
1 -> | 1 | 7 | <- 6
2 -> | 2 | 0 | <- 5
3 -> | 2 | 6 | <- 4
4 -> | 8 | 1 | <- 3
5 -> | 7 | 0 | <- 2
6 -> |_6_|_5_| <- 1
         1         
Turn: P1


Choose your move (1-6):  6


P1               P2
     ____2____     
1 -> | 1 | 7 | <- 6
2 -> | 2 | 1 | <- 5
3 -> | 2 | 7 | <- 4
4 -> | 8 | 2 | <- 3
5 -> | 7 | 1 | <- 2
6 -> |_0_|_6_| <- 1
         2         
Turn: P2
[1, 2, 2, 8, 7, 0, 2, 6, 1, 2, 7, 1, 7, 2]
Random Player chose pit: 4
P1               P2
     ____3____     
1 -> | 2 | 8 | <- 6
2 -> | 3 | 2 | <- 5
3 -> | 3 | 0 | <- 4
4 -> | 9 | 2 | <- 3
5 -> | 7 | 1 | <- 2
6 -> |_0_|_6_| <- 1
         2         
Turn: P1


Choose your move (1-6):  5


P1               P2
     ____3____     
1 -> | 2 | 8 | <- 6
2 -> | 3 | 3 | <- 5
3 -> | 3 | 1 | <- 4
4 -> | 9 | 3 | <- 3
5 -> | 0 | 2 | <- 2
6 -> |_1_|_7_| <- 1
         3         
Turn: P2
[2, 3, 3, 9, 0, 1, 3, 7, 2, 3, 1, 3, 8, 3]
Random Player chose pit: 5
P1               P2
     ____4____     
1 -> | 3 | 9 | <- 6
2 -> | 3 | 0 | <- 5
3 -> | 3 | 1 | <- 4
4 -> | 9 | 3 | <- 3
5 -> | 0 | 2 | <- 2
6 -> |_1_|_7_| <- 1
         3         
Turn: P1

List of moves played:
Player 1 selected pit 2
Player 2 selected pit 3
Player 1 selected pit 3
Player 2 selected pit 2
Player 1 selected pit 1
Player 2 selected pit 5
Player 1 selected pit 6
Player 2 selected pit 4
Player 1 selected pit 5
Player 2 selected pit 5


### Below is 100 Games of a Random Player vs another Random Player

In [16]:
def simulate_random_vs_random(num_games):
    p1_wins = 0
    p2_wins = 0
    total_moves = 0

    for game in range(num_games):
        game_instance = Mancala()
        moves_in_game = 0

        while not game_instance.winning_eval():
            # Player 1's random move
            random_move_p1 = game_instance.random_move_generator()
            if random_move_p1 is not None:
                game_instance.play(random_move_p1)
                moves_in_game += 1
            
            if game_instance.winning_eval():
                break

            # Player 2's random move
            random_move_p2 = game_instance.random_move_generator()
            if random_move_p2 is not None:
                game_instance.play(random_move_p2)
                moves_in_game += 1

        # Determine winner
        if game_instance.board[game_instance.p1_mancala_index] > game_instance.board[game_instance.p2_mancala_index]:
            p1_wins += 1
        elif game_instance.board[game_instance.p1_mancala_index] < game_instance.board[game_instance.p2_mancala_index]:
            p2_wins += 1

        total_moves += moves_in_game

    # Calculate results
    p1_win_percentage = (p1_wins / num_games) * 100
    p2_win_percentage = (p2_wins / num_games) * 100
    average_moves_per_game = total_moves / num_games

    return p1_win_percentage, p2_win_percentage, average_moves_per_game


# Simulate and display results
p1_win_percentage, p2_win_percentage, avg_moves = simulate_random_vs_random(num_games=100)
print(f"Player 1 win percentage: {p1_win_percentage}%")
print(f"Player 2 win percentage: {p2_win_percentage}%")
print(f"Average moves per game: {avg_moves}")

Player 1 win percentage: 47.0%
Player 2 win percentage: 45.0%
Average moves per game: 43.48


As we can see above, both Player 1's and Player 2's win percentage is 47.0% (averages around 44-52% for each player after multiple trials). The average moves per game is 43.46. It's important to mention that these numbers may fluctuate when running multiple times since they are random players, however, they generally fall close to the numbers listed.

### Below is the implementation of the AI Player using minimax

In [19]:
def utility_function(board, p1_mancala_index, p2_mancala_index, ai_player):
    if ai_player == 1:
        return board[p1_mancala_index] - board[p2_mancala_index]
    else:
        return board[p2_mancala_index] - board[p1_mancala_index]
    
def minimax(game, depth, maximizing_player):
    """Minimax algorithm for Mancala."""
    if depth == 0 or game.winning_eval():
        # Return utility value if at terminal state or depth limit
        return utility_function(
            game.board,
            game.p1_mancala_index,
            game.p2_mancala_index,
            maximizing_player
        )

    if maximizing_player == game.current_player:
        max_eval = float('-inf')
        best_move = None
        for move in range(1, game.pits_per_player + 1):
            if game.valid_move(move):
                # Simulate the move
                saved_board = game.board[:]
                saved_player = game.current_player
                game.play(move)

                # Recursively evaluate the move
                eval = minimax(game, depth - 1, maximizing_player)
                game.board = saved_board  # Undo the move
                game.current_player = saved_player

                if eval > max_eval:
                    max_eval = eval
                    best_move = move
        return best_move if depth == 5 else max_eval
    else:
        min_eval = float('inf')
        for move in range(1, game.pits_per_player + 1):
            if game.valid_move(move):
                # Simulate the move
                saved_board = game.board[:]
                saved_player = game.current_player
                game.play(move)

                # Recursively evaluate the move
                eval = minimax(game, depth - 1, maximizing_player)
                game.board = saved_board  # Undo the move
                game.current_player = saved_player

                min_eval = min(min_eval, eval)
        return min_eval


def ai_move(game, depth):
    return minimax(game, depth, game.current_player)

def simulate_ai_vs_random(num_games, depth):
    """
    Simulates games between the AI and a random player.
    """
    ai_wins = 0
    random_wins = 0
    total_moves = 0

    for game_num in range(num_games):
        game = Mancala()
        moves_in_game = 0

        while not game.winning_eval():
            if game.current_player == 1:  # AI's turn
                move = ai_move(game, depth)
            else:  # Random player's turn
                move = game.random_move_generator()

            if move is not None:
                game.play(move)
                moves_in_game += 1

        # Determine winner
        if game.board[game.p1_mancala_index] > game.board[game.p2_mancala_index]:
            ai_wins += 1
        elif game.board[game.p1_mancala_index] < game.board[game.p2_mancala_index]:
            random_wins += 1

        total_moves += moves_in_game

    # Calculate results
    ai_win_percentage = (ai_wins / num_games) * 100
    random_win_percentage = (random_wins / num_games) * 100
    average_moves = total_moves / num_games

    return ai_win_percentage, random_win_percentage, average_moves

In [26]:
# Simulate Minimax AI vs Random games
ai_win_percentage, random_win_percentage, avg_moves = simulate_ai_vs_random(100, 5)
print(f"AI win percentage: {ai_win_percentage}%")
print(f"Random win percentage: {random_win_percentage}%")
print(f"Average moves per game: {avg_moves}")

AI win percentage: 98.0%
Random win percentage: 1.0%
Average moves per game: 29.91


We notice above that the Minimax AI's win percentage is 98%, while the Random Player's win percentage is 1%, which is certainly better than the random player.. The average moves per game is 30.87, which is lower than the average for two random players as well. The AI that uses minimax is certainly a better alternative to the random player. We see that not only do we win at a much higher percentage than before (averaging around 98% with multiple trials), but we can also do it in fewer moves than the random players. Overall, this AI is not only better than the random player (as expected), but also wins faster and more efficiently.

### Below is the implementation of the AI Player using Alpha-Beta Pruning

In [21]:
import time

def alpha_beta(game, depth, maximizing_player, alpha=float('-inf'), beta=float('inf')):
    """Alpha-Beta pruning algorithm for Mancala."""
    if depth == 0 or game.winning_eval():
        return utility_function(
            game.board,
            game.p1_mancala_index,
            game.p2_mancala_index,
            maximizing_player
        )

    if maximizing_player == game.current_player:
        max_eval = float('-inf')
        best_move = None
        for move in range(1, game.pits_per_player + 1):
            if game.valid_move(move):
                # Simulate the move
                saved_board = game.board[:]
                saved_player = game.current_player
                game.play(move)

                # Evaluate the move recursively
                eval = alpha_beta(game, depth - 1, maximizing_player, alpha, beta)
                game.board = saved_board  # Undo the move
                game.current_player = saved_player

                if eval > max_eval:
                    max_eval = eval
                    best_move = move
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break
        return best_move if depth == 5 else max_eval
    else:
        min_eval = float('inf')
        for move in range(1, game.pits_per_player + 1):
            if game.valid_move(move):
                # Simulate the move
                saved_board = game.board[:]
                saved_player = game.current_player
                game.play(move)

                # Evaluate the move recursively
                eval = alpha_beta(game, depth - 1, maximizing_player, alpha, beta)
                game.board = saved_board  # Undo the move
                game.current_player = saved_player

                min_eval = min(min_eval, eval)
                beta = min(beta, eval)
                if beta <= alpha:
                    break
        return min_eval

def alpha_beta_move(game, depth=5):
    """AI chooses the best move using Alpha-Beta pruning."""
    return alpha_beta(game, depth, game.current_player)

def simulate_alpha_beta_vs_random(num_games, depth):
    """Simulate games between the Alpha-Beta AI and a random player."""
    ai_wins = 0
    random_wins = 0
    total_moves = 0
    total_time = 0

    for _ in range(num_games):
        game = Mancala()
        moves_in_game = 0
        start_time = time.time()

        while not game.winning_eval():
            if game.current_player == 1:  # AI's turn
                move = alpha_beta_move(game, depth)
            else:  # Random player's turn
                move = game.random_move_generator()

            if move is not None:
                game.play(move)
                moves_in_game += 1

        end_time = time.time()
        total_time += (end_time - start_time)

        # Determine winner
        if game.board[game.p1_mancala_index] > game.board[game.p2_mancala_index]:
            ai_wins += 1
        elif game.board[game.p1_mancala_index] < game.board[game.p2_mancala_index]:
            random_wins += 1

        total_moves += moves_in_game

    # Calculate results
    ai_win_percentage = (ai_wins / num_games) * 100
    random_win_percentage = (random_wins / num_games) * 100
    average_moves = total_moves / num_games
    average_time = total_time / num_games

    return ai_win_percentage, random_win_percentage, average_moves, average_time

In [25]:
# Simulate Alpha-Beta AI vs Random games
ai_win_percentage, random_win_percentage, avg_moves, avg_time = simulate_alpha_beta_vs_random(100, 5)
print(f"AI win percentage: {ai_win_percentage}%")
print(f"Random win percentage: {random_win_percentage}%")
print(f"Average moves per game: {avg_moves}")
print(f"Average time per game: {avg_time:.2f} seconds")

AI win percentage: 98.0%
Random win percentage: 1.0%
Average moves per game: 30.98
Average time per game: 0.02 seconds


A single game to run to completion takes about 0.02 seconds, which we were able to calculate by measuring the total time of the simulation using the time library, and dividing by the total amount of games played (100). We notice that the AI's win percentage is 98%, while the Random player's is only 1%. This is to be expected since the Alpha-Beta Pruning AI and Minimax AI are examining nearly identical trees in order to make their decisions. The only difference between the two is the speed in which the the AI's complete a game. Since the Alpha-Beta Pruning AI can bypass visiting certain branches of the search tree, it can complete a game in less time than Minimax. Overall, the performance of this AI is relatively similar to the Minimax AI, the only difference being the speed in which the Alpha-Beta AI can run.